In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
import re



In [2]:
# Read CSV file into a Pandas DataFrame
df_kaggle = pd.read_csv('../kaggle/the_oscar_award.csv')

# # Change column values based on conditions
# df_kaggle.loc[df_kaggle['Winner'] == 1, 'Winner'] = True
# df_kaggle.loc[df_kaggle['Winner'].isna(), 'Winner'] = False

#removing everything that appears in ()
#define a regular expression to match parentheses and their contents
pattern = r'\s*\([^)]*\)'

# remove parentheses and their contents from column 'A'
df_kaggle['category'] = df_kaggle['category'].replace(pattern, '', regex=True)

def clean_string(s):
    s= s.title()
    # use regular expressions to match words with multiple capital letters
    pattern = r'\b\w*[A-Z]+\w*\b'
    matches = re.findall(pattern, s)
    for match in matches:
        # convert the matched word to lowercase except for the first letter
        s = s.replace(match, match.capitalize())
    return s

df_kaggle['category'] = df_kaggle['category'].apply(clean_string)

In [3]:
# filter out rows containing 'Award' in the Category column
df_kaggle = df_kaggle[~df_kaggle['category'].str.contains('Award')]

#at first Best Picture Award was called Outstanding Picture and Unique and Artistic Picture
df_kaggle['category'] = df_kaggle['category'].replace(['Outstanding Picture', 'Unique And Artistic Picture', 'Outstanding Motion Picture', 'Best Motion Picture', 'Outstanding Production'], 'Best Picture')


#Actor award was changed to Actor in a Leading Role in 1976 I'm leaving it as Actor
df_kaggle['category'] = df_kaggle['category'].replace('Actor In A Leading Role', 'Actor')
df_kaggle['category'] = df_kaggle['category'].replace('Actress In A Leading Role', 'Actress')

# Best Art Direction changed to Best Production Design in 2012
df_kaggle['category'] = df_kaggle['category'].replace('Art Direction', 'Production Design')

df_kaggle['category'] = df_kaggle['category'].replace('Makeup', 'Makeup And Hairstyling')

df_kaggle['category'] = df_kaggle['category'].replace('Foreign Language Film', 'International Feature Film')

# create boolean mask for rows containing target strings
mask = df_kaggle['category'].str.contains('Sound')
# replace values in rows matching the mask with the desired string
df_kaggle.loc[mask, 'category'] = 'Sound'

# create boolean mask for rows containing target strings
mask2 = df_kaggle['category'].str.contains('Effects')
# replace values in rows matching the mask with the desired string
df_kaggle.loc[mask2, 'category'] = 'VFX'

# out of Documentary category in 2022 Documentary Feature Film and Documentary Short Film emerged - im changing it to documentary

df_kaggle['category'] = df_kaggle['category'].replace(['Documentary Short Film', 'Documentary Feature Film'], 'Documentary')




In [278]:
df_kaggle.head(5)

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,Actor,Richard Barthelmess,The Noose,False
1,1927,1928,1,Actor,Emil Jannings,The Last Command,True
2,1927,1928,1,Actress,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,Actress,Janet Gaynor,7th Heaven,True
4,1927,1928,1,Actress,Gloria Swanson,Sadie Thompson,False


In [4]:
#nominated and winners of best picture award Oscars from 1927 (year, title, wiki, winner(T/F))
osc_scrape = pd.read_csv('../data/scraping_results/osc_bp.csv')

# Filter the DataFrame based on a column value
ocs_winners = osc_scrape[osc_scrape['winner'] == True].drop('winner', axis=1)

In [5]:
ocs_winners

,year,film,wiki
0,1927,Wings,/wiki/Wings_(1927_film)
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody
8,1929,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(1930_film)
13,1930,Cimarron,/wiki/Cimarron_(1931_film)
18,1931,Grand Hotel,/wiki/Grand_Hotel_(1932_film)
...,...,...,...
546,2018,Green Book,/wiki/Green_Book_(film)
554,2019,Parasite,/wiki/Parasite_(2019_film)
563,2020,Nomadland,/wiki/Nomadland_(film)
571,2021,CODA,/wiki/CODA_(2021_film)


In [6]:
# I want to look only into winners nominations
df_noms = df_kaggle[df_kaggle['film'].isin(ocs_winners['film'])].copy()
df_year_noms = df_noms.merge(ocs_winners, left_on='year_film', right_on='year', how='inner')
df_noms_filtered = df_year_noms[df_year_noms['film_x'] == df_year_noms['film_y']].drop(['winner', 'year','name', 'film_y'], axis=1)
df_noms_filtered


,year_film,year_ceremony,ceremony,category,film_x,wiki
0,1928,1929,2,Actress,The Broadway Melody,/wiki/The_Broadway_Melody
1,1928,1929,2,Directing,The Broadway Melody,/wiki/The_Broadway_Melody
2,1928,1929,2,Best Picture,The Broadway Melody,/wiki/The_Broadway_Melody
3,1934,1935,7,Actor,It Happened One Night,/wiki/It_Happened_One_Night
4,1934,1935,7,Actress,It Happened One Night,/wiki/It_Happened_One_Night
...,...,...,...,...,...,...
366,2022,2023,95,Film Editing,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
367,2022,2023,95,Music,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
368,2022,2023,95,Music,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
369,2022,2023,95,Best Picture,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once


In [7]:
# group by Category and aggregate the Year column
df_cats = df_noms.groupby('category')['year_film'].agg(['min', 'max']).reset_index()

# rename columns
df_cats.columns = ['Category', 'Earliest Year', 'Latest Year']
df_cats = df_cats[df_cats['Latest Year']==2022].reset_index(drop=True)
df_cats

,Category,Earliest Year,Latest Year
0,Actor In A Supporting Role,1937,2022
1,Actress,1928,2022
2,Actress In A Supporting Role,1942,2022
3,Best Picture,1928,2022
4,Costume Design,1953,2022
5,Directing,1928,2022
6,Film Editing,1936,2022
7,Music,1937,2022
8,Writing,1934,2022


In [8]:
#function that keeps only the main categories of nominations
def main_noms(df, general_cats):
    df['category'] = df['category'].replace(['Actor', 'Actress', 'Actor In A Supporitng Role', 'Actress In A Supporitng Role'], 'Acting')
    df_main_noms = df[df['category'].isin(general_cats)]
    return df_main_noms


In [9]:
general_cats = ['Writing', 'Best Picture', 'Acting', 'Directing', 'Film Editing', 'Music', 'Cinematography', 'Sound', 'Production Design', 'Costume', 'Makeup and Hairstyling', 'VFX']
# im creating a separate df which consists of generalised nominations - all acting into one, got rid of animation, international, short and documenatry
df_main_noms=main_noms(df_noms_filtered, general_cats)
df_main_noms

,year_film,year_ceremony,ceremony,category,film_x,wiki
0,1928,1929,2,Acting,The Broadway Melody,/wiki/The_Broadway_Melody
1,1928,1929,2,Directing,The Broadway Melody,/wiki/The_Broadway_Melody
2,1928,1929,2,Best Picture,The Broadway Melody,/wiki/The_Broadway_Melody
3,1934,1935,7,Acting,It Happened One Night,/wiki/It_Happened_One_Night
4,1934,1935,7,Acting,It Happened One Night,/wiki/It_Happened_One_Night
...,...,...,...,...,...,...
366,2022,2023,95,Film Editing,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
367,2022,2023,95,Music,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
368,2022,2023,95,Music,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
369,2022,2023,95,Best Picture,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once


In [10]:
nominations = df_main_noms.drop_duplicates()
nominations

,year_film,year_ceremony,ceremony,category,film_x,wiki
0,1928,1929,2,Acting,The Broadway Melody,/wiki/The_Broadway_Melody
1,1928,1929,2,Directing,The Broadway Melody,/wiki/The_Broadway_Melody
2,1928,1929,2,Best Picture,The Broadway Melody,/wiki/The_Broadway_Melody
3,1934,1935,7,Acting,It Happened One Night,/wiki/It_Happened_One_Night
5,1934,1935,7,Directing,It Happened One Night,/wiki/It_Happened_One_Night
...,...,...,...,...,...,...
365,2022,2023,95,Directing,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
366,2022,2023,95,Film Editing,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
367,2022,2023,95,Music,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once
369,2022,2023,95,Best Picture,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once


In [12]:

n_of_ceremonies = nominations.iloc[-1, 2]
# count the occurrences of each value in the list
value_counts = nominations['category'].value_counts(normalize=False).to_frame().reset_index()


value_counts.columns= ['category','number']
value_counts['percentage'] = (value_counts['number']/n_of_ceremonies).apply(lambda x: '{:.2f}%'.format(x*100))
print(value_counts)
value_counts.to_csv('./nominations.csv', index = False)

            category  number percentage
0       Best Picture      40     42.11%
1          Directing      39     41.05%
2            Writing      39     41.05%
3       Film Editing      34     35.79%
4             Acting      33     34.74%
5              Music      24     25.26%
6              Sound      22     23.16%
7     Cinematography      21     22.11%
8  Production Design      17     17.89%
9                VFX       3      3.16%


In [13]:
#nomintions of the best picture nominees in 2022/2023
df_2022 = df_kaggle[df_kaggle['year_film']==2022]
nominees = ['Everything Everywhere All at Once', 'All uiet on the Western Front', 'Avatar: The Way of Water', 'The Banshees of Inisherin', 'Elvis', 'The Fablemans', 'Tár', 'Top Gun: Maverick', 'Triangle of Sadness', 'Women Talking']
df_2022_bp = df_2022[df_2022['film'].isin(nominees)].reset_index(drop=True).drop(['year_film', 'year_ceremony', 'ceremony'], axis=1)
df_2022_main_noms = main_noms(df_2022_bp, general_cats)
df_2022_main_noms

,category,name,film,winner
0,Acting,Austin Butler,Elvis,False
1,Acting,Colin Farrell,The Banshees of Inisherin,False
5,Acting,Cate Blanchett,Tár,False
6,Acting,Michelle Yeoh,Everything Everywhere All at Once,True
10,Cinematography,Mandy Walker,Elvis,False
11,Cinematography,Florian Hoffmeister,Tár,False
14,Directing,Martin McDonagh,The Banshees of Inisherin,False
15,Directing,Daniel Kwan and Daniel Scheinert,Everything Everywhere All at Once,True
16,Directing,Todd Field,Tár,False
17,Directing,Ruben Östlund,Triangle of Sadness,False


In [22]:
nominations_df = pd.DataFrame(index=nominees, columns=general_cats)
for film in nominees:
    for category in general_cats:
        if not pd.isna(nominations_df.loc[film, category]):
            continue
        if not df_2022_main_noms[(df_2022_main_noms['category'] == category) & (df_2022_main_noms['film'] == film)].empty:
            nominations_df.loc[film, category] = 'Won' if df_2022_main_noms[(df_2022_main_noms['category'] == category) & (df_2022_main_noms['film'] == film)]['winner'].values[0] else 'Nominated'
        else:
            nominations_df.loc[film, category] = 'Not nominated'
nominations_df
nominations_df.to_csv('./nom_win_df.csv')


In [ ]:
import tabulate
from tabulate import tabulate
import pyperclip
# Zamiana wartości w komórkach na kropki i puste komórki
nominations_df.replace({'Won' : '..', 'Nominated': '•', 'Not nominated': ' '}, inplace=True)

# Konwersja DataFrame na tablicę
table = tabulate(nominations_df, headers='keys', tablefmt='grid')
# Copy the table to the clipboard
pyperclip.copy(table)

# Wyświetlenie tabeli
print(table)

In [274]:
# list of current noms
cats_soup = BeautifulSoup(rq.get('https://en.wikipedia.org/wiki/Academy_Awards').text, 'lxml')

# find all caption tags that contain the text 'Population'
caption_tag = cats_soup.find('caption', text=re.compile("List of current Awards of Merit categories by year introduced, sortable by category"))

cats_table = caption_tag.find_parent('table')
print(type(cats_table))
# use the pandas 'read_html()' function to convert the table to a DataFrame
cats_df = pd.read_html(str(cats_table))[0]
categories= cats_df['Category'].values
# print the DataFrame
print(categories)


<class 'bs4.element.Tag'>
['Best Picture' 'Best Director' 'Best Actor' 'Best Actress'
 'Best Cinematography' 'Best Production Design' 'Best Adapted Screenplay'
 'Best Sound' 'Best Animated Short Film' 'Best Live Action Short Film'
 'Best Film Editing' 'Best Original Score' 'Best Original Song'
 'Best Supporting Actor' 'Best Supporting Actress' 'Best Visual Effects'
 'Best Original Screenplay' 'Best Documentary Short Film'
 'Best Documentary Feature Film' 'Best International Feature Film'
 'Best Costume Design' 'Best Makeup and Hairstyling'
 'Best Animated Feature Film']


so oscar categories names changed throughout the years
honorary award is a separate to oscars award given by the board of governors - celbrates acheivements not covered by exisitng academy awards - im dleeting it